Sentiment Analysis

In [1]:
import tensorflow as tf
from datasets import load_dataset
from transformers import BertTokenizer
from einops import rearrange, repeat
import matplotlib.pyplot as plt

2024-04-11 20:44:25.827856: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 20:44:25.912016: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 20:44:26.281940: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 20:44:27.224633: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = load_dataset("sst", "default")

In [3]:
# longest sentence 267 -> max seqzence lengtha and input ? -> padding
train_data = dataset["train"]["sentence"]
# labels are floats between 0 and 1, need to be rounded to zero or 1 for sentiment classification
train_labels = dataset["train"]["label"]
print(len(train_data))

val_data = dataset["validation"]["sentence"]
# labels are floats between 0 and 1, need to be rounded to zero or 1 for sentiment classification
val_labels = dataset["validation"]["label"]
print(len(val_data))


8544
1101


In [4]:

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
vocab_size = tokenizer.vocab_size

In [5]:
def data_preprocessing(data, targets, tokenizer, max_length):
  tokenized_data = []
  for input in data:
      t = tokenizer.encode_plus(text=input, max_length=max_length, padding="max_length")
      tokenized_data.append(t["input_ids"])
  binary_targets = tf.math.round(targets)
  dataset = tf.data.Dataset.from_tensor_slices((tokenized_data, binary_targets))
  dataset = dataset.shuffle(1000).batch(32).prefetch(4)
  return dataset

train_dataset = data_preprocessing(train_data, train_labels, tokenizer, 267)
validation_dataset = data_preprocessing(val_data, val_labels, tokenizer, 267)

In [6]:
class MambaResBlock(tf.keras.Model):

  def __init__(self, input_dim):
    super().__init__()

    self.input_dim = input_dim
    self.projection_dim = 2*input_dim

    # normalisation
    self.layernorm = tf.keras.layers.LayerNormalization()
    # Dense
    self.dense1 = tf.keras.layers.Dense(units=self.projection_dim)
    # Dense
    self.dense2 = tf.keras.layers.Dense(units=self.projection_dim)
    # Convolution
    self.conv1d = tf.keras.layers.Conv1D(filters=self.projection_dim , kernel_size=4, strides=1 , padding="causal", groups = self.projection_dim, data_format = "channels_last") # data_format?, groups?
    # SSM block
    self.ssm = SelectiveSSM(32, self.projection_dim)
    # Dense
    self.dense3 = tf.keras.layers.Dense(units=input_dim)
    # dropout
    self.dropout = tf.keras.layers.Dropout(rate=0.2)

  # forward step
  def call(self, input):

    x = self.layernorm(input)

    x1 = self.dense1(x)
    x1 = self.conv1d(x1)
    x1 =  tf.nn.silu(x1)
    x1 = self.ssm(x1)

    x2 = self.dense2(x)
    x2 =  tf.nn.silu(x2)

    x = x1 * x2
    x = self.dense3(x)

    # skip connection
    x = x + input

    x = self.dropout(x)

    return x

class SelectiveSSM(tf.keras.Model):
  def __init__(self, states, projection_dim):
    super().__init__()

    self.states = states
    self.projection_dim = projection_dim

    # hippo initialisation für A ? dafür müsste A aber quadratisch sein
    # -> quadratische matrix oder nicht ?
    #self.A =  # states x internal dim
    #self.D =  # np ones internal dim
    A = repeat(tf.range(1, states+1, dtype=tf.float32),'n -> d n', d=self.projection_dim)

    # A parameter: matrix changing hidden state
    self.A_log = tf.Variable(tf.math.log(A),trainable=True, dtype=tf.float32)

    # D parameter: simple skip connection, initialisde as ones
    self.D = tf.Variable(tf.ones(projection_dim),trainable=True, dtype=tf.float32) # change from np to tf

    # B, C, delta parameter: dependend on input, therefore dense layer to learn these
    self.denseB = tf.keras.layers.Dense(units=self.states)
    self.denseC = tf.keras.layers.Dense(units=self.states)
    self.densedelta = tf.keras.layers.Dense(units=self.projection_dim)

  def selective_scan(self,input, delta, A, B, C, D):
    """
    Calculate output of the selective state space model using parallel scan
    implemented using the cumulative sum

    args:
      input: data input that we calculate the ssm on
      delta: mediates how much focus is put on new input
      A: state matrix controlling the hidden state
      B: modulate the recurrent dynamics based on content (input)
      C: modulate the recurrent dynamics based on context (hidden states)
      D: scales the skip connection

    returns:
      output: result of the ssm with current parameters

    """
    # first step of discretization of A
    deltaA = tf.einsum('bld,dn->bldn', delta, A) # quasi delta mal A
    deltaBinput = tf.einsum('bld,bld,bln->bldn', delta, input, B) # input mal B mal delta

    deltaA_cumsum = tf.pad(
        deltaA[:, 1:], [[0, 0], [1, 1], [0, 0], [0, 0]])[:, 1:, :, :]

    deltaA_cumsum = tf.reverse(deltaA_cumsum, axis=[1])  # Flip along axis 1

    # Cumulative sum along all the input tokens, parallel prefix sum,
    # calculates dA for all the input tokens in parallel
    deltaA_cumsum = tf.math.cumsum(deltaA_cumsum, axis=1)

    # second step of discretization of A
    deltaA_cumsum = tf.exp(deltaA_cumsum)
    deltaA_cumsum = tf.reverse(deltaA_cumsum, axis=[1])  # Flip back along axis 1

    # calculate intermediate output as in graphs shown for ssm's
    x = deltaBinput * deltaA_cumsum
    # 1e-12 to avoid division by 0
    x = tf.math.cumsum(x, axis=1)/(deltaA_cumsum + 1e-12)

    # intermediate output multiplied with parameter C
    output = tf.einsum('bldn,bln->bld', x, C)

    return output + input * D

  def call(self, input):

    A = -tf.exp(tf.cast(self.A_log, tf.float32))

    C = self.denseC(input)
    B = self.denseB(input)
    # softplus to not get nan values
    delta = tf.nn.softplus(self.densedelta(input))

    return self.selective_scan(input, delta, A, B, C, self.D)

class MambaModel(tf.keras.Model):
  def __init__(self, num_layers, vocab_size, input_dim):
    super().__init__()

    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = input_dim) #, input_length = 128) (bs, 128, 128)
    self.layer_list = []
    for i in range(num_layers):
        self.layer_list.append(MambaResBlock(input_dim))
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(units=1024, activation=tf.nn.gelu)
    self.out = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)

  def call(self, input):

    x = self.embedding(input)

    for i in range(self.num_layers):
      x = self.layer_list[i](x)

    x = self.flatten(x)
    x = self.dense(x)
    x = self.out(x)

    return x

In [7]:
model = MambaModel(3, vocab_size, input_dim=267)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()

# compile the model (here, adding a loss function and an optimizer)
model.compile(optimizer = optimizer, loss=loss, metrics=["accuracy"])


In [ ]:
history = model.fit(train_dataset,validation_data=validation_dataset, epochs=10)

In [ ]:
def visualise_results(history):

  plt.plot(history.history["loss"], label="train loss")
  plt.plot(history.history["val_loss"], label="validation loss")
  plt.title("Train and Validation Loss")
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()

  plt.plot(history.history["accuracy"], label="train accuracy")
  plt.plot(history.history["val_accuracy"], label="validation accuracy")
  plt.title("Train and Validation Accuracy")
  plt.xlabel("Epoch")
  plt.ylabel("Accuracy")
  plt.legend()
  plt.show()

visualise_results(history)